In [ ]:
import os


In [2]:
%pwd


'c:\\Github\\Text-Summarization\\research'

In [3]:
os.chdir("c:\\Github\\Text-Summarization")
%pwd


'c:\\Github\\Text-Summarization'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm


ImportError: cannot import name 'load_metric' from 'datasets' (c:\Users\X1 Yoga\.conda\envs\mlproj\lib\site-packages\datasets\__init__.py)

In [ ]:
from dataclasses import dataclass
from pathlib import Path


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    # Evaluation

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list-of_elements."""
        for i in range(0, len(self.list_of_elements), self.batch_size):
            yield self.list_of_elements[i:i+self.batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16,

                                    device=device, column_text="article",
                                    column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(
            dataset[self.column_text], self.batch_size))
        target_batches = list(generate_batch_sized_chunks(
            dataset[self.column_summary], self.batch_size))

        for article_batch, target_batch in tqdm(
                zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,
                               truncation=True, padding="max_length", return_tensors='pt')

            summaries = model.generate(input_ids=inputs['input-ids'].to(device),
                                       attention_mask=inputs['attention_mask'].to(
                device),
                length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that th model does not genrate sequences that are too long'''

            # Finally, we decode the generated texts,
            # replace the token, and add the decoded texts with the references to the metric.

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True) for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,
                             references=target_batch)

            # Finally compute and return the ROUGE scores.

        score = metric.compute()
        return score


In [ ]:
import evaluate
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# rouge_metric = load_metric('rouge')
metric = evaluate.load("rouge")


In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text="dialogue", column_summary="summary")
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f'pegasus'])


In [ ]:
# try:
#     config = ConfigurationManager()
#     model_evaluation_config = config.get_model_evaluation_config()
#     model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
#     model_evaluation_config.train()
# except Exception as e:
#     raise e
